# Reto 1 - Orientado a un Proyecto

# Contentido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Visión general de los datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores faltantes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)

## Introducción <a id='intro'></a>
Cuando estamos investigando, necesitamos formular hipótesis que luego podemos poner a prueba. A veces aceptamos estas hipótesis; otras veces las rechazamos. Para tomar decisiones acertadas, una empresa debe poder comprender si está haciendo las suposiciones correctas.

En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de Yandex.Music para probar las siguientes hipótesis y comparar el comportamiento de los usuarios en estas dos ciudades.

### Objetivo: 
Poner a prueba tres hipótesis:
1. La actividad de los usuarios difiere según el día de la semana y la ciudad.
2. Los residentes de Springfield y Shelbyville escuchan géneros musicales diferentes los lunes por la mañana. Esto también es cierto para las tardes de los viernes.
3. Los oyentes de Springfield y Shelbyville tienen preferencias diferentes. En Springfield prefieren el pop, mientras que en Shelbyville hay más fanáticos del rap.

### Etapas 
Los datos sobre el comportamiento de los usuarios se encuentran en el archivo `/data/music.csv`. No hay información sobre la calidad de los datos, por lo que deberás explorarlos antes de probar las hipótesis.

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Luego, durante el preprocesamiento de datos, intentarás solucionar los problemas más críticos.

Tu proyecto constará de tres etapas:
1. Visión general de los datos
2. Preprocesamiento de datos
3. Pruebas de las hipótesis

[Tabla de Contenido](#back)

## Etapa 1. Visión general de los datos <a id='data_review'></a>

Importar los paquetes necesarios:

In [1]:
import pandas as pd

Read the file `music_project_en.csv` from the `/datasets/` folder and save it in the `df` variable:

In [2]:
# reading the file and storing it to df
df = pd.read_csv('data/music.csv')

Obtener información sobre los 10 primeros registros:

In [3]:
# Obteniendo las primeras 10 filas de la tabla df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Obteniendo información general sobre los datos en df

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Todas ellas almacenan el mismo tipo de datos: `object`.

Según la documentación:
- `'userID'` — identificador de usuario
- `'Track'` — título de la canción
- `'artist'` — nombre del artista
- `'genre'` — género musical
- `'City'` — ciudad del usuario
- `'time'` — hora exacta en que se reprodujo la canción
- `'Day'` — día de la semana

Podemos observar tres problemas con el estilo de los nombres de las columnas:
1. Algunos nombres están en mayúsculas y otros en minúsculas.
2. Hay espacios en algunos nombres.
3. `Detecta el tercer problema tú mismo y descríbelo aquí`.



### Conclusiones <a id='data_review_conclusions'></a> 

Cada fila en la tabla almacena datos sobre una canción que fue reproducida. Algunas columnas describen la canción en sí: su título, artista y género. El resto transmiten información sobre el usuario: la ciudad de la que provienen y la hora en que reprodujeron la canción.

Es evidente que los datos son suficientes para probar las hipótesis. Sin embargo, existen valores faltantes.

Para avanzar, necesitamos preprocesar los datos.

[Tabla de Contenido](#back)

## Etapa 2. Pre-Procesamiento de Datos <a id='data_preprocessing'></a>
Correct the formatting in the column headers and deal with the missing values. Then, check whether there are duplicates in the data.

### Header style <a id='header_style'></a>
Print the column header:

In [5]:
# the list of column names in the df table
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Cambiar los nombres de columna de acuerdo a las reglas de estilo adecuado:

    Si el nombre tiene varias palabras, utilizar snake_case
    Todos los caracteres deben estar en minúscula
    Eliminar espacios

In [6]:
df = df.rename(columns={'  userID': 'user_id',
                        'Track': 'track',
                        '  City  ': 'city',
                        'Day': 'day'})

Verifica el resultado. Imprime los nombres de las columnas una vez más:

In [7]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Back to Contents](#back)

### Valores Nulos <a id='missing_values'></a>
Primero, encuentra el número de valores faltantes en la tabla. Para hacerlo, utiliza dos métodos de pandas encadenados:

In [8]:
df.isna().sum()

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

No todos los valores faltantes afectan la investigación. Por ejemplo, los valores faltantes en `track` y `artist` no son críticos. Simplemente puedes reemplazarlos con marcadores claros.

Sin embargo, los valores faltantes en `'genre'` pueden afectar la comparación de las preferencias musicales en Springfield y Shelbyville. En la vida real, sería útil conocer las razones por las cuales faltan los datos e intentar compensarlos. Pero no tenemos esa oportunidad en este proyecto. Por lo tanto, deberás:
* Rellenar estos valores faltantes con marcadores
* Evaluar cuánto pueden afectar los valores faltantes a tus cálculos


Reemplaza los valores faltantes en `'track'`, `'artist'` y `'genre'` con la cadena de texto `'unknown'`. Para hacer esto, crea la lista `columns_to_replace`, recórrela con un bucle `for` y reemplaza los valores faltantes en cada una de las columnas:


In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for col in columns_to_replace:
  df[col] = df[col].fillna('unknown')

Asegúrate de que la tabla no contenga más valores faltantes. Vuelve a contar los valores faltantes.

In [10]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Tabla de Contenido](#back)

### Duplicados <a id='duplicates'></a>
Encuentra el número de duplicados obvios en la tabla utilizando dos comandos encadenados:

In [11]:
df.duplicated().sum()

3826

Llama al método de pandas para eliminar los duplicados obvios:

In [12]:
df = df.drop_duplicates().reset_index(drop=True)

Vuelve a contar los duplicados obvios para asegurarte de haber eliminado todos ellos:

In [13]:
df.duplicated().sum()

0

Ahora elimina los duplicados implícitos en la columna `'genre'`. Por ejemplo, el nombre de un género puede estar escrito de diferentes maneras. Estos errores también afectarán el resultado.

Imprime una lista de nombres de género únicos, ordenados alfabéticamente. Para hacerlo:

- Obtén la columna deseada del DataFrame
- Aplica un método de ordenamiento a esa columna
- Para la columna ordenada, llama al método que devolverá todos los valores únicos de la columna

In [14]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Revisa la lista para encontrar duplicados implícitos del género `hiphop`. Estos podrían ser nombres escritos incorrectamente o nombres alternativos del mismo género.

Verás los siguientes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para deshacerte de ellos, declara la función `replace_wrong_genres()` con dos parámetros:
* `wrong_genres=`: la lista de duplicados
* `correct_genre=`: la cadena con el valor correcto

La función debe corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, reemplazar cada valor de la lista `wrong_genres` con el valor en `correct_genre`.


In [15]:
# function for replacing implicit duplicates

def replace_wrong_genres(wrong_genres, correct_genre):
  for wrong_genre in wrong_genres:
    df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

Llama a `replace_wrong_genres()` y pásale argumentos para que elimine los duplicados implícitos (`hip`, `hop` y `hip-hop`) y los reemplace con `hiphop`:


In [16]:
# removing implicit duplicates
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'

replace_wrong_genres(wrong_genres, correct_genre)

Asegúrate de que los nombres duplicados hayan sido eliminados. Imprime la lista de valores únicos de la columna `'genre'`:


In [17]:
# checking for implicit duplicates
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

[Tabla de Contenido](#back)

### Conclusiones <a id='data_preprocessing_conclusions'></a>

Hemos detectado tres problemas con los datos:

- Estilo incorrecto de encabezados
- Valores faltantes
- Duplicados obvios e implícitos

Los encabezados han sido corregidos para facilitar el procesamiento de la tabla.

Todos los valores faltantes han sido reemplazados por `'unknown'`. Sin embargo, aún debemos ver si los valores faltantes en `'genre'` afectarán nuestros cálculos.

La ausencia de duplicados hará que los resultados sean más precisos y más fáciles de entender.

Ahora podemos pasar a probar las hipótesis.


[Tabla de Contenidos](#back)